In [1]:
import pandas as pd
import numpy as np
import ast

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [22]:

# from mysql.connector import connect

# con = connect(host='178.250.242.127', user='dan_m', password='EuHQuyzD]3iw6rvm', database='b1978_news')
# cur = con.cursor()

# cur.execute("SELECT * FROM dataset WHERE ticker LIKE 'BA' AND tickersMentioned IS NOT NULL ORDER BY tickersMentioned ASC")
# df = pd.DataFrame(cur.fetchall(), columns=cur.column_names).dropna(subset=['tickersMentioned'])
# con.close()

# df.head()

,id,article_id,date,ticker,publisher,url,title,content,tickersMentioned,tags,polarity_eod,neg_eod,neu_eod,pos_eod,neg,neu,pos
0,2627872,"[140, 159, 124, 177, 11, 238, 162, 56, 102, 51...",2024-01-10,BA,New York Post,https://nypost.com/2024/01/10/business/boeing-...,Boeing CEO fights back tears as he admits to ‘...,Boeing's CEO Dave Calhoun fought back tears wh...,nan,nan,NaN,NaN,NaN,NaN,0.717140,0.277656,0.005204
1,2627871,"[195, 152, 251, 63, 246, 189, 112, 121, 38, 61...",2024-01-10,BA,Yahoo Finance,https://www.youtube.com/watch?v=kUWRMnRndIg,Boeing CEO acknowledges 'mistake' regarding 73...,The Federal Aviation Administration (FAA) insp...,nan,nan,NaN,NaN,NaN,NaN,0.394112,0.604134,0.001754
2,2627870,"[195, 166, 102, 188, 114, 45, 14, 125, 160, 7,...",2024-01-10,BA,Reuters,https://www.reuters.com/business/aerospace-def...,Boeing's latest MAX problem creates more heada...,"A cabin blowout at 16,000 feet and the groundi...",nan,nan,NaN,NaN,NaN,NaN,0.997366,0.001434,0.001200
3,2627869,"[1, 230, 103, 113, 157, 68, 84, 189, 70, 3, 1,...",2024-01-10,BA,CNBC Television,https://www.youtube.com/watch?v=fvRqD7ChfAQ,Boeing 737 Max-9 investigation: What you need ...,CNBC's Phil LeBeau joins 'Halftime Report' to ...,nan,,NaN,NaN,NaN,NaN,0.000930,0.998989,0.000081
4,2627873,"[142, 226, 142, 255, 156, 105, 153, 86, 118, 1...",2024-01-10,BA,Forbes,https://www.forbes.com/sites/greatspeculations...,Will Boeing Stock Recover To Its Highs Of Over...,Boeing stock (NYSE: BA) currently trades at $2...,nan,nan,NaN,NaN,NaN,NaN,0.997857,0.001002,0.001141


In [31]:
df_val = pd.read_csv('BA_full_val.csv')
df_val = df_val[['date', 'title', 'content', 'tickersMentioned']]
df_val = df_val.dropna(subset=['tickersMentioned'])
df_val['tickersMentioned'] = df_val['tickersMentioned'].apply(eval)
df_val

,date,title,content,tickersMentioned
0,2023-07-24,Trending tickers: Ryanair | Vodafone | Ocado,Ryanair nearly quadrupled its profit from the ...,"[$ANRX-USD.CC, $COIN-USD.CC, 0OC.F, 0XMR-USD.C..."
1,2023-10-06,Boeing Won't Take Flight Until the Dollar Lose...,There is a strong inverse relationship between...,"[$KING-USD.CC, $ROPE-USD.CC, 0XMR-USD.CC, 1MT-..."
2,2024-01-09,LIVE: FTSE 100 higher amid Wall Street tech rally,The FTSE bucked the trend across Europe to tra...,"[$ROPE-USD.CC, 005930.KO, 005935.KO, 0R01.LSE,..."
3,2023-02-09,UPDATE 1-Lessor Avolon sees sharp air travel r...,"(Adds details, quotes)\n\nBy Tim Hepher\n\nPAR...","[000415.SHE, AIR.F, AIR.MC, AIR.MI, AIR.PA, AI..."
4,2023-12-12,UPDATE 1-Avolon agrees $18 bln deal for 140 Ai...,"(Adds details, quotes in paragraphs 4-9)\n\nDU...","[000415.SHE, AIR.F, AIR.MC, AIR.PA, AIR.XETRA,..."
...,...,...,...,...
4072,2022-03-21,A Boeing 737 Crashed in China. It Wasn’t a MAX...,While the aircraft wasn't a MAX version of Boe...,[BA.US]
4073,2023-02-21,"Boeing Stock Is a Buy, But Only if This Happens","BA stock recently traded near its highs, but i...",[BA.US]
4074,2023-03-14,BA Stock Alert: What to Know About Boeing&#039...,Boeing (BA) stock is on the move Tuesday follo...,[BA.US]
4075,2021-06-28,Why Boeing Stock Is Under Pressure Today,FAA Reportedly Tells Boeing That 777X Would No...,[BA.US]


In [32]:
data_tickers = pd.read_csv('all_tickers.csv')
data_tickers = data_tickers[['ticker', 'company']]
data_tickers

,ticker,company
0,MAPT4.SA,cemepe investimentos s.a.
1,CPLE3.SA,companhia paranaense de energia - copel
2,CPLE6.SA,companhia paranaense de energia - copel
3,POM.TO,polymet mining corp.
4,PTM.TO,platinum group metals ltd.
...,...,...
52015,ELC,"entergy louisiana, llc collateral tr mt"
52016,WRLGF,west red lake gold mines ltd.
52017,TUXS,tuxis corporation
52018,GHTI,g-h-3 international inc.


In [33]:
tickers = set(data_tickers.ticker.tolist())

In [34]:
df_val['tickersMentioned'] = df_val['tickersMentioned'].apply(lambda x: set(x) & tickers)
df_val = df_val.loc[df_val['tickersMentioned'] != set()]
# df_val['tickersMentionedCount'] = df_val['tickersMentioned'].apply(len)
df_val

,date,title,content,tickersMentioned
3,2023-02-09,UPDATE 1-Lessor Avolon sees sharp air travel r...,"(Adds details, quotes)\n\nBy Tim Hepher\n\nPAR...",{AIR.PA}
4,2023-12-12,UPDATE 1-Avolon agrees $18 bln deal for 140 Ai...,"(Adds details, quotes in paragraphs 4-9)\n\nDU...",{AIR.PA}
13,2022-06-01,Solar Giants’ Fates May Hinge on a Fight Over ...,(Bloomberg) -- Two Chinese solar giants are in...,{AIR.PA}
15,2024-01-09,"We didn't mean to rip people off at the pump, ...",Asda was accused by MPs of being 'one of the l...,"{MSF.F, FB2A.F}"
16,2023-10-11,Weight-Loss Drugs Have Finally Arrived. Why th...,Birkenstock set to begin trading after $1.5 bi...,{SSUN.F}
...,...,...,...,...
3908,2021-09-21,GOL Plans To Launch Network Of 250 eVTOL Aircr...,"GOL Linhas Aereas Inteligentes SA (NYSE: GOL),...",{GOLL4.SA}
3923,2021-02-24,UPDATE 1-Boeing not engaging with Norwegian Ai...,"(Adds details, no immediate comment from Boein...",{NAS.OL}
3924,2021-02-24,Norwegian Air says Boeing not engaging with Ir...,DUBLIN (Reuters) - Boeing is not engaging with...,{NAS.OL}
3925,2022-05-30,"Norwegian Air orders 50 Boeing MAX aircraft, o...",By Terje Solsvik\n\nOSLO (Reuters) -Norwegian ...,{NAS.OL}


## Загружаем модель и распознаем сущности

In [37]:
from span_marker import SpanMarkerModel

model = SpanMarkerModel.from_pretrained("lxyuan/span-marker-bert-base-multilingual-uncased-multinerd")

description = "Singapore is renowned for its hawker centers offering dishes \
like Hainanese chicken rice and laksa, while Malaysia boasts dishes such as \
nasi lemak and rendang, reflecting its rich culinary heritage."

entities = model.predict(description)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [38]:
# Функция, чтобы отфильтровать только организации
def filter_organizations(entities):
    return [entity for entity in entities if 'ORG' in entity['label']]

def find_entity(example):
    predict = model.predict(example)
    org_entities = filter_organizations(predict)

    stack = []
    # Пройдемся по списку, избегая выхода за пределы индекса
    for i in range(len(org_entities)):
        stack.append(org_entities[i]['span'])
    return stack

In [40]:
df_val['entities'] = np.nan

/var/folders/yj/nhkf3nv505d5kh0dsk42f3ym0000gn/T/ipykernel_29566/3000491649.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val['entities'] = np.nan


In [42]:
df_val['entities'] = df_val['content'].apply(find_entity)

/var/folders/yj/nhkf3nv505d5kh0dsk42f3ym0000gn/T/ipykernel_29566/1366979200.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val['entities'] = df_val['content'].apply(find_entity)


In [43]:
df_val

,date,title,content,tickersMentioned,entities
3,2023-02-09,UPDATE 1-Lessor Avolon sees sharp air travel r...,"(Adds details, quotes)\n\nBy Tim Hepher\n\nPAR...",{AIR.PA},"[Reuters, Avolon, Reuters, International Civil..."
4,2023-12-12,UPDATE 1-Avolon agrees $18 bln deal for 140 Ai...,"(Adds details, quotes in paragraphs 4-9)\n\nDU...",{AIR.PA},"[Reuters, Avolon, Airbus, Boeing, Bohai Leasin..."
13,2022-06-01,Solar Giants’ Fates May Hinge on a Fight Over ...,(Bloomberg) -- Two Chinese solar giants are in...,{AIR.PA},"[Bloomberg, Bloomberg, Tesla Execs, JPMorgan, ..."
15,2024-01-09,"We didn't mean to rip people off at the pump, ...",Asda was accused by MPs of being 'one of the l...,"{MSF.F, FB2A.F}","[Asda, REUTERS, TDR Capital, TDR]"
16,2023-10-11,Weight-Loss Drugs Have Finally Arrived. Why th...,Birkenstock set to begin trading after $1.5 bi...,{SSUN.F},[Birkenstock]
...,...,...,...,...,...
3908,2021-09-21,GOL Plans To Launch Network Of 250 eVTOL Aircr...,"GOL Linhas Aereas Inteligentes SA (NYSE: GOL),...",{GOLL4.SA},"[GOL Linhas Aereas Inteligentes SA, Grupo Comp..."
3923,2021-02-24,UPDATE 1-Boeing not engaging with Norwegian Ai...,"(Adds details, no immediate comment from Boein...",{NAS.OL},"[Boeing, Reuters, Boeing, Norwegian Air, Boein..."
3924,2021-02-24,Norwegian Air says Boeing not engaging with Ir...,DUBLIN (Reuters) - Boeing is not engaging with...,{NAS.OL},"[Reuters, Boeing, Norwegian Air]"
3925,2022-05-30,"Norwegian Air orders 50 Boeing MAX aircraft, o...",By Terje Solsvik\n\nOSLO (Reuters) -Norwegian ...,{NAS.OL},"[Terje Solsvik\n\nOSLO, Reuters, Norwegian Air..."


In [45]:
df_val['entities'].iloc[0]

['Reuters',
 'Avolon',
 'Reuters',
 'International Civil Aviation Organization',
 'International Air Transport Association']

In [47]:
data_tickers.head(3)

,ticker,company
0,MAPT4.SA,cemepe investimentos s.a.
1,CPLE3.SA,companhia paranaense de energia - copel
2,CPLE6.SA,companhia paranaense de energia - copel


In [56]:
df_val['entities'].iloc[1]

['Reuters',
 'Avolon',
 'Airbus',
 'Boeing',
 'Bohai Leasing Co Ltd',
 'Shenzhen Stock Exchange',
 'Bohai']

In [58]:
data_tickers[data_tickers['ticker'] == 'AIR.PA']

,ticker,company
12508,AIR.PA,airbus se


In [59]:
data_tickers_name = data_tickers['company'].to_list()
data_tickers_name

['cemepe investimentos s.a.',
 'companhia paranaense de energia - copel',
 'companhia paranaense de energia - copel',
 'polymet mining corp.',
 'platinum group metals ltd.',
 'minupar participaáıes s.a.',
 's„o martinho s.a.',
 'ultrapar participaáıes s.a.',
 'petrûleo brasileiro s.a. - petrobras',
 'cielo s.a.',
 'natura &co holding s.a.',
 'unipar carbocloro s.a.',
 'banco bradesco s.a.',
 'banco bradesco s.a.',
 'weg s.a.',
 'fleury s.a.',
 'cyrela brazil realty s.a. empreendimentos e participaáıes',
 'inepar s.a. ind˙stria e construáıes',
 'orizon valorizaá„o de resìduos s.a.',
 'mèliuz s.a.',
 'ccr s.a.',
 'b3 s.a. - brasil, bolsa, balc„o',
 'ita˙sa s.a.',
 'americanas s.a.',
 'locaweb serviáos de internet s.a.',
 'moura dubeux engenharia s.a.',
 'sendas distribuidora s.a.',
 'cogna educaá„o s.a.',
 'cia de ferro ligas da bahia - ferbasa',
 'yduqs participaáıes s.a.',
 'log-in logìstica intermodal s.a.',
 'boa vista serviáos s.a.',
 'braskem s.a.',
 'iguatemi s.a.',
 'centro de im

In [65]:
df_val['tickers'] = np.nan
df_val

/var/folders/yj/nhkf3nv505d5kh0dsk42f3ym0000gn/T/ipykernel_29566/167585129.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val['tickers'] = np.nan


,date,title,content,tickersMentioned,entities,tickers
3,2023-02-09,UPDATE 1-Lessor Avolon sees sharp air travel r...,"(Adds details, quotes)\n\nBy Tim Hepher\n\nPAR...",{AIR.PA},"[Reuters, Avolon, Reuters, International Civil...",NaN
4,2023-12-12,UPDATE 1-Avolon agrees $18 bln deal for 140 Ai...,"(Adds details, quotes in paragraphs 4-9)\n\nDU...",{AIR.PA},"[Reuters, Avolon, Airbus, Boeing, Bohai Leasin...",NaN
13,2022-06-01,Solar Giants’ Fates May Hinge on a Fight Over ...,(Bloomberg) -- Two Chinese solar giants are in...,{AIR.PA},"[Bloomberg, Bloomberg, Tesla Execs, JPMorgan, ...",NaN
15,2024-01-09,"We didn't mean to rip people off at the pump, ...",Asda was accused by MPs of being 'one of the l...,"{MSF.F, FB2A.F}","[Asda, REUTERS, TDR Capital, TDR]",NaN
16,2023-10-11,Weight-Loss Drugs Have Finally Arrived. Why th...,Birkenstock set to begin trading after $1.5 bi...,{SSUN.F},[Birkenstock],NaN
...,...,...,...,...,...,...
3908,2021-09-21,GOL Plans To Launch Network Of 250 eVTOL Aircr...,"GOL Linhas Aereas Inteligentes SA (NYSE: GOL),...",{GOLL4.SA},"[GOL Linhas Aereas Inteligentes SA, Grupo Comp...",NaN
3923,2021-02-24,UPDATE 1-Boeing not engaging with Norwegian Ai...,"(Adds details, no immediate comment from Boein...",{NAS.OL},"[Boeing, Reuters, Boeing, Norwegian Air, Boein...",NaN
3924,2021-02-24,Norwegian Air says Boeing not engaging with Ir...,DUBLIN (Reuters) - Boeing is not engaging with...,{NAS.OL},"[Reuters, Boeing, Norwegian Air]",NaN
3925,2022-05-30,"Norwegian Air orders 50 Boeing MAX aircraft, o...",By Terje Solsvik\n\nOSLO (Reuters) -Norwegian ...,{NAS.OL},"[Terje Solsvik\n\nOSLO, Reuters, Norwegian Air...",NaN


In [70]:
df_val['entities'].iloc[1]

['Reuters',
 'Avolon',
 'Airbus',
 'Boeing',
 'Bohai Leasing Co Ltd',
 'Shenzhen Stock Exchange',
 'Bohai']

In [69]:
extraction_results = process.extract(str(df_val['entities'].iloc[1]), data_tickers_name, limit=5)
extraction_results

matching_companies = [company for company, score in extraction_results if score >= 87]
matching_companies

AttributeError: 'list' object has no attribute 'lower'

In [71]:
list_entities = [i.lower() for i in df_val['entities'].iloc[0]]
list_entities

['reuters',
 'avolon',
 'reuters',
 'international civil aviation organization',
 'international air transport association']

In [74]:
for index in range(len(df_val)):
    # Выполняем fuzzy matching для извлечения потенциальных компаний
    list_entities = [i.lower() for i in df_val['entities'].iloc[index] ]
    extraction_results = process.extract(str(list_entities), data_tickers_name, limit=5)

    # Фильтруем компании со score >= 87
    matching_companies = [company for company, score in extraction_results if score >= 86]

    if matching_companies:
        # Получаем коды компаний, соответствующие matching_companies
        matching_codes = [data_tickers[data_tickers['company'] == company]['ticker'].values[0] for company in matching_companies]

        codes_str = ", ".join(matching_codes)

        df_val['tickers'].iloc[index] = codes_str

/var/folders/yj/nhkf3nv505d5kh0dsk42f3ym0000gn/T/ipykernel_29566/2998448435.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val['tickers'].iloc[index] = codes_str


In [75]:
df_val

,date,title,content,tickersMentioned,entities,tickers
3,2023-02-09,UPDATE 1-Lessor Avolon sees sharp air travel r...,"(Adds details, quotes)\n\nBy Tim Hepher\n\nPAR...",{AIR.PA},"[Reuters, Avolon, Reuters, International Civil...","MEAL3.SA, WAB, SEE, IP, RHI"
4,2023-12-12,UPDATE 1-Avolon agrees $18 bln deal for 140 Ai...,"(Adds details, quotes in paragraphs 4-9)\n\nDU...",{AIR.PA},"[Reuters, Avolon, Airbus, Boeing, Bohai Leasin...","PTM.TO, NTCO3.SA, TFCO4.SA, WTM, TEL"
13,2022-06-01,Solar Giants’ Fates May Hinge on a Fight Over ...,(Bloomberg) -- Two Chinese solar giants are in...,{AIR.PA},"[Bloomberg, Bloomberg, Tesla Execs, JPMorgan, ...","NTCO3.SA, TFCO4.SA, DTE, D(exp@2024-01-13), MRK"
15,2024-01-09,"We didn't mean to rip people off at the pump, ...",Asda was accused by MPs of being 'one of the l...,"{MSF.F, FB2A.F}","[Asda, REUTERS, TDR Capital, TDR]","RITM-PD, OWL, BRSP, DX, LADR"
16,2023-10-11,Weight-Loss Drugs Have Finally Arrived. Why th...,Birkenstock set to begin trading after $1.5 bi...,{SSUN.F},[Birkenstock],NaN
...,...,...,...,...,...,...
3908,2021-09-21,GOL Plans To Launch Network Of 250 eVTOL Aircr...,"GOL Linhas Aereas Inteligentes SA (NYSE: GOL),...",{GOLL4.SA},"[GOL Linhas Aereas Inteligentes SA, Grupo Comp...","GOLL4.SA, GOLL4.SA, SOMA3.SA, SBFG3.SA, KRSA3.SA"
3923,2021-02-24,UPDATE 1-Boeing not engaging with Norwegian Ai...,"(Adds details, no immediate comment from Boein...",{NAS.OL},"[Boeing, Reuters, Boeing, Norwegian Air, Boein...","WAB, SEE, APD, AL, BA"
3924,2021-02-24,Norwegian Air says Boeing not engaging with Ir...,DUBLIN (Reuters) - Boeing is not engaging with...,{NAS.OL},"[Reuters, Boeing, Norwegian Air]","SEE, AL, BA, DAL, MESA"
3925,2022-05-30,"Norwegian Air orders 50 Boeing MAX aircraft, o...",By Terje Solsvik\n\nOSLO (Reuters) -Norwegian ...,{NAS.OL},"[Terje Solsvik\n\nOSLO, Reuters, Norwegian Air...","WAB, SEE, LYB, ZGN, APD"


In [77]:
df_val.reset_index()

,index,date,title,content,tickersMentioned,entities,tickers
0,3,2023-02-09,UPDATE 1-Lessor Avolon sees sharp air travel r...,"(Adds details, quotes)\n\nBy Tim Hepher\n\nPAR...",{AIR.PA},"[Reuters, Avolon, Reuters, International Civil...","MEAL3.SA, WAB, SEE, IP, RHI"
1,4,2023-12-12,UPDATE 1-Avolon agrees $18 bln deal for 140 Ai...,"(Adds details, quotes in paragraphs 4-9)\n\nDU...",{AIR.PA},"[Reuters, Avolon, Airbus, Boeing, Bohai Leasin...","PTM.TO, NTCO3.SA, TFCO4.SA, WTM, TEL"
2,13,2022-06-01,Solar Giants’ Fates May Hinge on a Fight Over ...,(Bloomberg) -- Two Chinese solar giants are in...,{AIR.PA},"[Bloomberg, Bloomberg, Tesla Execs, JPMorgan, ...","NTCO3.SA, TFCO4.SA, DTE, D(exp@2024-01-13), MRK"
3,15,2024-01-09,"We didn't mean to rip people off at the pump, ...",Asda was accused by MPs of being 'one of the l...,"{MSF.F, FB2A.F}","[Asda, REUTERS, TDR Capital, TDR]","RITM-PD, OWL, BRSP, DX, LADR"
4,16,2023-10-11,Weight-Loss Drugs Have Finally Arrived. Why th...,Birkenstock set to begin trading after $1.5 bi...,{SSUN.F},[Birkenstock],NaN
...,...,...,...,...,...,...,...
657,3908,2021-09-21,GOL Plans To Launch Network Of 250 eVTOL Aircr...,"GOL Linhas Aereas Inteligentes SA (NYSE: GOL),...",{GOLL4.SA},"[GOL Linhas Aereas Inteligentes SA, Grupo Comp...","GOLL4.SA, GOLL4.SA, SOMA3.SA, SBFG3.SA, KRSA3.SA"
658,3923,2021-02-24,UPDATE 1-Boeing not engaging with Norwegian Ai...,"(Adds details, no immediate comment from Boein...",{NAS.OL},"[Boeing, Reuters, Boeing, Norwegian Air, Boein...","WAB, SEE, APD, AL, BA"
659,3924,2021-02-24,Norwegian Air says Boeing not engaging with Ir...,DUBLIN (Reuters) - Boeing is not engaging with...,{NAS.OL},"[Reuters, Boeing, Norwegian Air]","SEE, AL, BA, DAL, MESA"
660,3925,2022-05-30,"Norwegian Air orders 50 Boeing MAX aircraft, o...",By Terje Solsvik\n\nOSLO (Reuters) -Norwegian ...,{NAS.OL},"[Terje Solsvik\n\nOSLO, Reuters, Norwegian Air...","WAB, SEE, LYB, ZGN, APD"


In [102]:
df_val['tickersMentioned'] = df_val['tickersMentioned'].apply(lambda x: list(x))

/var/folders/yj/nhkf3nv505d5kh0dsk42f3ym0000gn/T/ipykernel_29566/1065129811.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val['tickersMentioned'] = df_val['tickersMentioned'].apply(lambda x: list(x))


In [103]:
df_val

,date,title,content,tickersMentioned,entities,tickers
3,2023-02-09,UPDATE 1-Lessor Avolon sees sharp air travel r...,"(Adds details, quotes)\n\nBy Tim Hepher\n\nPAR...",[AIR.PA],"[Reuters, Avolon, Reuters, International Civil...","MEAL3.SA, WAB, SEE, IP, RHI"
4,2023-12-12,UPDATE 1-Avolon agrees $18 bln deal for 140 Ai...,"(Adds details, quotes in paragraphs 4-9)\n\nDU...",[AIR.PA],"[Reuters, Avolon, Airbus, Boeing, Bohai Leasin...","PTM.TO, NTCO3.SA, TFCO4.SA, WTM, TEL"
13,2022-06-01,Solar Giants’ Fates May Hinge on a Fight Over ...,(Bloomberg) -- Two Chinese solar giants are in...,[AIR.PA],"[Bloomberg, Bloomberg, Tesla Execs, JPMorgan, ...","NTCO3.SA, TFCO4.SA, DTE, D(exp@2024-01-13), MRK"
15,2024-01-09,"We didn't mean to rip people off at the pump, ...",Asda was accused by MPs of being 'one of the l...,"[MSF.F, FB2A.F]","[Asda, REUTERS, TDR Capital, TDR]","RITM-PD, OWL, BRSP, DX, LADR"
16,2023-10-11,Weight-Loss Drugs Have Finally Arrived. Why th...,Birkenstock set to begin trading after $1.5 bi...,[SSUN.F],[Birkenstock],NaN
...,...,...,...,...,...,...
3908,2021-09-21,GOL Plans To Launch Network Of 250 eVTOL Aircr...,"GOL Linhas Aereas Inteligentes SA (NYSE: GOL),...",[GOLL4.SA],"[GOL Linhas Aereas Inteligentes SA, Grupo Comp...","GOLL4.SA, GOLL4.SA, SOMA3.SA, SBFG3.SA, KRSA3.SA"
3923,2021-02-24,UPDATE 1-Boeing not engaging with Norwegian Ai...,"(Adds details, no immediate comment from Boein...",[NAS.OL],"[Boeing, Reuters, Boeing, Norwegian Air, Boein...","WAB, SEE, APD, AL, BA"
3924,2021-02-24,Norwegian Air says Boeing not engaging with Ir...,DUBLIN (Reuters) - Boeing is not engaging with...,[NAS.OL],"[Reuters, Boeing, Norwegian Air]","SEE, AL, BA, DAL, MESA"
3925,2022-05-30,"Norwegian Air orders 50 Boeing MAX aircraft, o...",By Terje Solsvik\n\nOSLO (Reuters) -Norwegian ...,[NAS.OL],"[Terje Solsvik\n\nOSLO, Reuters, Norwegian Air...","WAB, SEE, LYB, ZGN, APD"


In [110]:
df_val['tickers'] = df_val['tickers'].apply(lambda x: list(x.split(', ')) if isinstance(x, str) else [])


/var/folders/yj/nhkf3nv505d5kh0dsk42f3ym0000gn/T/ipykernel_29566/1264002875.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val['tickers'] = df_val['tickers'].apply(lambda x: list(x.split(', ')) if isinstance(x, str) else [])


In [123]:
df_val = df_val.reset_index()

In [125]:
df_val = df_val.drop(columns=['index'])

In [126]:
df_val

,date,title,content,tickersMentioned,entities,tickers
0,2023-02-09,UPDATE 1-Lessor Avolon sees sharp air travel r...,"(Adds details, quotes)\n\nBy Tim Hepher\n\nPAR...",[AIR.PA],"[Reuters, Avolon, Reuters, International Civil...","[MEAL3.SA, WAB, SEE, IP, RHI]"
1,2023-12-12,UPDATE 1-Avolon agrees $18 bln deal for 140 Ai...,"(Adds details, quotes in paragraphs 4-9)\n\nDU...",[AIR.PA],"[Reuters, Avolon, Airbus, Boeing, Bohai Leasin...","[PTM.TO, NTCO3.SA, TFCO4.SA, WTM, TEL]"
2,2022-06-01,Solar Giants’ Fates May Hinge on a Fight Over ...,(Bloomberg) -- Two Chinese solar giants are in...,[AIR.PA],"[Bloomberg, Bloomberg, Tesla Execs, JPMorgan, ...","[NTCO3.SA, TFCO4.SA, DTE, D(exp@2024-01-13), MRK]"
3,2024-01-09,"We didn't mean to rip people off at the pump, ...",Asda was accused by MPs of being 'one of the l...,"[MSF.F, FB2A.F]","[Asda, REUTERS, TDR Capital, TDR]","[RITM-PD, OWL, BRSP, DX, LADR]"
4,2023-10-11,Weight-Loss Drugs Have Finally Arrived. Why th...,Birkenstock set to begin trading after $1.5 bi...,[SSUN.F],[Birkenstock],[]
...,...,...,...,...,...,...
657,2021-09-21,GOL Plans To Launch Network Of 250 eVTOL Aircr...,"GOL Linhas Aereas Inteligentes SA (NYSE: GOL),...",[GOLL4.SA],"[GOL Linhas Aereas Inteligentes SA, Grupo Comp...","[GOLL4.SA, GOLL4.SA, SOMA3.SA, SBFG3.SA, KRSA3..."
658,2021-02-24,UPDATE 1-Boeing not engaging with Norwegian Ai...,"(Adds details, no immediate comment from Boein...",[NAS.OL],"[Boeing, Reuters, Boeing, Norwegian Air, Boein...","[WAB, SEE, APD, AL, BA]"
659,2021-02-24,Norwegian Air says Boeing not engaging with Ir...,DUBLIN (Reuters) - Boeing is not engaging with...,[NAS.OL],"[Reuters, Boeing, Norwegian Air]","[SEE, AL, BA, DAL, MESA]"
660,2022-05-30,"Norwegian Air orders 50 Boeing MAX aircraft, o...",By Terje Solsvik\n\nOSLO (Reuters) -Norwegian ...,[NAS.OL],"[Terje Solsvik\n\nOSLO, Reuters, Norwegian Air...","[WAB, SEE, LYB, ZGN, APD]"


In [143]:
counter = 0
for i in range(len(df_val)):
    if len(df_val['tickersMentioned'].iloc[i]) == 1:
        if df_val['tickersMentioned'].iloc[i][0] in df_val['tickers'].iloc[i]:
            counter += 1
    else:
        for ticker in df_val['tickersMentioned'].iloc[i]:
            if ticker in df_val['tickers'].iloc[i]:
                counter += 1


## Всего вхождений

In [144]:
counter

26

## Метрика правильно выявленных тикеров

In [145]:
counter / len(df_val)

0.03927492447129909